In [266]:
import pandas as pd
import numpy as np

In [267]:
# CS513 - Theory Of Data Cleaning - Final Project
### Description:
#### Cleaning Craigslist listings of rental apartments in Chicago
#### By:
#### - Kushagra Soni
#### - Peri Rocha

## Reads the dataset from the project-dataset-final.xlsx file and captures it into a Pandas Dataframe. 
data_file = "data/project-dataset-final.xlsx"
df_listings = pd.read_excel(data_file, index_col=None)
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   web-scraper-order      2858 non-null   object 
 1   web-scraper-start-url  2858 non-null   object 
 2   pagination             2751 non-null   object 
 3   listing-title          2858 non-null   object 
 4   listing-description    2858 non-null   object 
 5   listing-housing-type   2858 non-null   object 
 6   listing-features       2858 non-null   object 
 7   listing-notices        2858 non-null   object 
 8   listing-id             2858 non-null   object 
 9   listing-link           2858 non-null   object 
 10  listing-link-href      2858 non-null   object 
 11  listing-availability   1534 non-null   object 
 12  listing-posting-date   2858 non-null   object 
 13  listing-address        2098 non-null   object 
 14  listing-map-latitude   2857 non-null   float64
 15  list

In [268]:
## Get the listing-description fields from the data and Clean it
### - Removes the Prefix - "QR Code Link to This Post"
### - Removes line breaks and Extra Tabs and spaces and replaces them with semicolon ";"

import re

#### Creates regex patterns for newline, tabs and semicolon
regex_newline = re.compile(r'\n', flags=re.IGNORECASE)
regex_tab = re.compile(r'(\s{2,})', flags=re.IGNORECASE)
regex_semicolon = re.compile(r';+', flags=re.IGNORECASE)

#### Apply the regex(s) to the "listing-description" field and generate new field "description"
series_description = df_listings['listing-description'].str.replace(regex_newline, ';', regex=True)

series_description = series_description.str.replace(regex_tab, '', regex=True)
series_description = series_description.str.replace(regex_semicolon, ';', regex=True)
regex_qr_code = re.compile(r'QR Code Link to This Post;', flags=re.IGNORECASE)
series_description = series_description.str.replace(regex_qr_code, '', regex=True)
series_description = series_description.str.strip()
series_description = series_description.dropna(how='all', axis=0)
df_description = pd.DataFrame()
df_description['Description'] = series_description
df_description.info()
df_description

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2858 entries, 0 to 2857
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Description  2858 non-null   object
dtypes: object(1)
memory usage: 44.7+ KB


,Description
0,Apartment home available July 1st on Chicago’s...
1,Call Apartment Guys at 773-549-3474 and ask fo...
2,Contact me to learn more about an innovative L...
3,"PROPERTY INFO;ID: 232839975Rent: $3,500 / Mont..."
4,"4535 N Paulina Unit #3D Chicago, IL 60640;Brig..."
...,...
2853,Interested in this property?;Click on: Reply t...
2854,"We are conveniently located near Edens, Metra ..."
2855,Interested in this property?;Click on: Reply t...
2856,"PROPERTY INFO;ID: 237708192Rent: $3,147 / Mont..."


In [269]:
## Get the listing-title fields from the data and Clean it
### - Removes Rent Amount, BedRoom, Footage, etc. from beginning and creates individual fields listing-rent, listing-room-type and listing-footage
### - Note that when it says "0BR" on listing-housing-type, the title will not have the count of bedrooms

df_title = pd.DataFrame(df_listings['listing-title'])
####### listing_title.info()
####### listing_title


#### Creates regex patterns for Rent amount and create dataframe listing-rent
regex_rent = re.compile(r'(\$\d+\,*\d+\s)', flags=re.IGNORECASE)

#### Apply the regex(s) to the "listing-title" field and generate new field "description"
df_rent = df_title['listing-title'].str.extract(regex_rent, expand=True)
df_rent.columns = ['Rent']
df_rent['Rent'] = df_rent['Rent'].str.replace('\,', '')
df_rent.info()

#### Creates regex patterns for Room Type and create dataframe listing-room-type
# It checks for either 1 or more Bed Room / Studio type
regex_room_type = re.compile(r'(\d*br\s|studio|\d+\s*bed)', flags=re.IGNORECASE)

# Apply the regex(s) to the "listing-title" field and generate new field "housing type"
df_housing_type = df_listings['listing-title'].str.extract(regex_room_type, expand=True)
df_housing_type.columns = ['Housing Type']
df_housing_type.info()
df_housing_type['Housing Type'] = df_housing_type['Housing Type'].str.replace('br', ' Bed Room')
df_housing_type['Housing Type'] = df_housing_type['Housing Type'].str.replace(r'(?i)studio', 'STUDIO')
df_housing_type.info()

#### Creates regex patterns for footage and create dataframe listing-footage
regex_footage = re.compile(r'(\d+\s*(?=ft))', flags=re.IGNORECASE)

# Apply the regex(s) to the "listing-title" field and generate new field "description"
df_footage = df_listings['listing-title'].str.extract(regex_footage, expand=True)
df_footage.columns = ['Footage (sq-ft)']
df_footage.info()
df_footage
# df = df_footage.join(df_rent).join(df_room_type)
# df
df_footage.isna().sum()
df_footage.count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rent    2857 non-null   object
dtypes: object(1)
memory usage: 22.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Housing Type  2768 non-null   object
dtypes: object(1)
memory usage: 22.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Housing Type  2768 non-null   object
dtypes: object(1)
memory usage: 22.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0 

/var/folders/9t/fx1d82y17ld65ll42bk27zz40000gn/T/ipykernel_14461/1437155251.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rent['Rent'] = df_rent['Rent'].str.replace('\,', '')
/var/folders/9t/fx1d82y17ld65ll42bk27zz40000gn/T/ipykernel_14461/1437155251.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df_housing_type['Housing Type'] = df_housing_type['Housing Type'].str.replace(r'(?i)studio', 'STUDIO')


Footage (sq-ft)    1034
dtype: int64

In [270]:
## Get the listing-features fields from the data and Clean it
### - Separates the various features of the listing into individual columns

df_features = pd.DataFrame(df_listings['listing-features'])
# df_features.info()

#### Creates regex patterns for various features

# LISTING TYPE
regex_listing_type = re.compile(r'(house|apartment|duplex|flat|condo|townhouse|loft)|(?:open house)', flags=re.IGNORECASE)

# CATs Allowed
regex_cats_allowed = re.compile(r'(cats are OK)', flags=re.IGNORECASE)

# Dogs Allowed
regex_dogs_allowed = re.compile(r'(dogs are OK)', flags=re.IGNORECASE)

# LAUNDRY TYPE
regex_laundry = re.compile(r'(w/d in unit|w/d hookups|laundry in bldg|no laundry on site|laundry on site)', flags=re.IGNORECASE)

# GARAGE
regex_parking = re.compile(r'(attached garage|detached garage|no parking|off-street parking|street parking|carport|valet parking)', flags=re.IGNORECASE)

# RENT PERIOD
regex_rent_period = re.compile(r'(monthly|weekly)', flags=re.IGNORECASE)

# SMOKING
regex_smoking = re.compile(r'(\w+.smoking)', flags=re.IGNORECASE)

# APPLICATION FEES
regex_app_fees = re.compile(r'(\$\d+)', flags=re.IGNORECASE)


df_listing_type = df_features['listing-features'].str.extract(regex_listing_type)
df_listing_type.columns = ['Listing Type']

df_cats = pd.DataFrame(df_features['listing-features'].str.contains(regex_cats_allowed, na = False, regex=True))
df_cats.columns=['Cats Allowed']
df_cats['Cats Allowed'] = np.where(df_cats['Cats Allowed'], 'Y', 'N')

df_dogs = pd.DataFrame(df_features['listing-features'].str.contains(regex_dogs_allowed, na = False, regex=True))
df_dogs.columns=['Dogs Allowed']
df_dogs['Dogs Allowed'] = np.where(df_dogs['Dogs Allowed'], 'Y', 'N')

df_laundry = df_features['listing-features'].str.extract(regex_laundry, expand=True)
df_laundry.columns = ['Laundry Type']

df_parking = df_features['listing-features'].str.extract(regex_parking, expand=True)
df_parking.columns = ['Parking Type']

df_rent_period = df_features['listing-features'].str.extract(regex_rent_period, expand=True)
df_rent_period.columns = ['Rent Period']

df_smoking = pd.DataFrame(df_features['listing-features'].str.contains(regex_smoking, regex=True))
df_smoking.columns = ['No Smoking']
df_smoking['No Smoking'] = np.where(df_smoking['No Smoking'], 'Y', 'N')

df_app_fees = df_features['listing-features'].str.extract(regex_app_fees, expand=True)
df_app_fees.columns = ['Application Fees']
########### np.where(df_app_fees[0].isna() == True)[0]
df_smoking

/var/folders/9t/fx1d82y17ld65ll42bk27zz40000gn/T/ipykernel_14461/891142336.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_cats = pd.DataFrame(df_features['listing-features'].str.contains(regex_cats_allowed, na = False, regex=True))
/var/folders/9t/fx1d82y17ld65ll42bk27zz40000gn/T/ipykernel_14461/891142336.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_dogs = pd.DataFrame(df_features['listing-features'].str.contains(regex_dogs_allowed, na = False, regex=True))
/var/folders/9t/fx1d82y17ld65ll42bk27zz40000gn/T/ipykernel_14461/891142336.py:54: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_smoking = pd.DataFrame(df_features['listing-features'].str.contains(regex_smoking, regex=True))


,No Smoking
0,N
1,N
2,N
3,Y
4,Y
...,...
2853,N
2854,N
2855,N
2856,N


In [271]:
## Get the listing-housing-type fields from the data and Clean it
### - Separate Number of BedRooms and Bathrooms of the listing into individual columns

df_housing_type = pd.DataFrame(df_listings['listing-housing-type'])
df_housing_type.columns = ['listing-housing-type']

# BED ROOMS
regex_bedrooms = re.compile(r'(\d+(?=br))', flags=re.IGNORECASE)

# BATH
regex_baths = re.compile(r'(\d*\.*\d*(?=ba))', flags=re.IGNORECASE)

df_bedrooms = df_housing_type['listing-housing-type'].str.extract(regex_bedrooms, expand=True)
df_baths = df_housing_type['listing-housing-type'].str.extract(regex_baths, expand=True)


df_bedrooms.columns = ['Total Bed Rooms']
df_baths.columns = ['Total Baths']


In [272]:
## Get the images of the listing s and count the total images available

df_listing_link = pd.DataFrame(df_listings['listing-link'])
df_listing_link


# Regex to get total images

regex_images = re.compile(r'((?<=of)\s\d+)', flags=re.IGNORECASE)

df_images = df_listing_link['listing-link'].str.extract(regex_images, expand=True)
df_images.columns = ['Total Images']
df_images.isna().sum()
df_images.count()

Total Images    2794
dtype: int64

In [273]:
df_date_posted = pd.DataFrame(df_listings['listing-posting-date'])
df_date_posted

regex_date = re.compile(r'(\d+(?=\s*days ago)|day ago|hours ago|hour ago)', flags=re.IGNORECASE)

df_date_posted = df_date_posted['listing-posting-date'].str.extract(regex_date, expand=True)
df_date_posted.columns = ['Days Ago']
# df_date_posted['Posted Date'].value_counts()
df_date_posted['Days Ago'] = df_date_posted['Days Ago'].str.replace('day ago', '1')
df_date_posted['Days Ago'] = df_date_posted['Days Ago'].str.replace(r'hours ago', '1')
df_date_posted['Days Ago'] = df_date_posted['Days Ago'].astype(int)
df_date_posted['Fetch Date'] = pd.to_datetime('2016-11-06')
df_date_posted['Posted Date'] = df_date_posted['Fetch Date'] - pd.to_timedelta(df_date_posted['Days Ago'], unit='d')
df_date_posted

,Days Ago,Fetch Date,Posted Date
0,10,2016-11-06,2016-10-27
1,5,2016-11-06,2016-11-01
2,10,2016-11-06,2016-10-27
3,12,2016-11-06,2016-10-25
4,4,2016-11-06,2016-11-02
...,...,...,...
2853,3,2016-11-06,2016-11-03
2854,5,2016-11-06,2016-11-01
2855,8,2016-11-06,2016-10-29
2856,12,2016-11-06,2016-10-25


In [274]:
df_kush = df_description.join(df_rent).join(df_footage).join(df_listing_type).join(df_cats).join(df_dogs).join(df_laundry).join(df_parking).join(df_rent_period).join(df_smoking).join(df_app_fees).join(df_bedrooms).join(df_baths).join(df_images).join(df_date_posted['Posted Date'])
## df_description.to_excel("data/description.xlsx", index=False)
df_kush

,Description,Rent,Footage (sq-ft),Listing Type,Cats Allowed,Dogs Allowed,Laundry Type,Parking Type,Rent Period,No Smoking,Application Fees,Total Bed Rooms,Total Baths,Total Images,Posted Date
0,Apartment home available July 1st on Chicago’s...,$1795,NaN,apartment,Y,N,w/d in unit,attached garage,monthly,N,NaN,1,1,19,2016-10-27
1,Call Apartment Guys at 773-549-3474 and ask fo...,$1650,NaN,apartment,Y,Y,w/d in unit,street parking,monthly,N,NaN,1,1,10,2016-11-01
2,Contact me to learn more about an innovative L...,$2860,2213,House,Y,Y,w/d hookups,attached garage,monthly,N,$90,4,2.5,NaN,2016-10-27
3,"PROPERTY INFO;ID: 232839975Rent: $3,500 / Mont...",$3500,NaN,apartment,Y,Y,w/d in unit,attached garage,monthly,Y,NaN,2,2,23,2016-10-25
4,"4535 N Paulina Unit #3D Chicago, IL 60640;Brig...",$1225,NaN,apartment,Y,Y,laundry in bldg,street parking,monthly,Y,NaN,1,1,8,2016-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853,Interested in this property?;Click on: Reply t...,$3685,NaN,apartment,Y,Y,w/d in unit,attached garage,monthly,N,NaN,2,2,13,2016-11-03
2854,"We are conveniently located near Edens, Metra ...",$1400,NaN,apartment,N,N,laundry on site,off-street parking,monthly,N,NaN,1,1,11,2016-11-01
2855,Interested in this property?;Click on: Reply t...,$2250,NaN,apartment,Y,Y,w/d in unit,attached garage,monthly,N,NaN,1,1,6,2016-10-29
2856,"PROPERTY INFO;ID: 237708192Rent: $3,147 / Mont...",$3147,NaN,apartment,Y,Y,laundry in bldg,attached garage,monthly,N,NaN,3,2,9,2016-10-25
